<a href="https://colab.research.google.com/github/SolKidonakis/TPFINAL_NLP/blob/main/TPFINAL_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -U langchain langchain-community openai pandas neo4j xmltodict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall

In [9]:
import fitz  # PyMuPDF para trabajar con PDFs
import pandas as pd
from langchain.document_loaders import PubMedLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from neo4j import GraphDatabase


In [36]:
import fitz  # Importar PyMuPDF para trabajar con PDFs

# Función para extraer el contenido de los documentos PDF
def extract_text_from_pdf(pdf_file):
    try:
        doc = fitz.open(pdf_file)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error al abrir el archivo PDF {pdf_file}: {e}")
        return ""

# Nombres de los archivos PDF
pdf_files = ['Argentina.pdf', 'Brasil.pdf']

# Extraer el texto de cada PDF
pdf_texts = {}
for pdf_file in pdf_files:
    pdf_texts[pdf_file] = extract_text_from_pdf(pdf_file)

# Mostrar los primeros caracteres de cada texto (opcional)
for pdf_file, text in pdf_texts.items():
    print(f"Texto extraído de {pdf_file}:")
    print(text[:500])  # Mostrar solo los primeros 500 caracteres
    print()

# Dividir el texto en fragmentos de longitud específica
chunk_size = 1000
processed_texts = {}
for pdf_file, text in pdf_texts.items():
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    processed_texts[pdf_file] = chunks

# Mostrar los primeros fragmentos de texto procesados (opcional)
for pdf_file, chunks in processed_texts.items():
    print(f"Fragmentos procesados de {pdf_file}:")
    for chunk in chunks[:3]:  # Mostrar solo los primeros 3 fragmentos
        print(chunk[:200])  # Mostrar solo los primeros 200 caracteres
    print()


Texto extraído de Argentina.pdf:
República Argentinaa ​
Bandera
Escudo
Lema: En Unión y Libertadb ​
Himno: Himno Nacional Argentino
¿Problemas al reproducir este archivo?
Territorio controlado por la República Argentina en verde
oscuro; territorio reclamado pero no controlado en verde
claro.
Capital
(y ciudad más
poblada)
Buenos Aires
34°35′59″S 58°22′55″O
Idioma oficial
Español1 ​
 • Cooficiales
Guaraní (cooficial en
Corrientes)
Qom, mocoví y wichí
(cooficial en Chaco)
 • Hablados
Lenguas de Argentina
Gentilicio
Argentino, -a


Texto extraído de Brasil.pdf:
República Federativa de Brasil
República Federativa do
Brasil  (portugués)
Bandera
Escudo
Lema: «Ordem e progresso»
(en español: «Orden y progreso»)
Himno: Hino Nacional Brasileiro
(en portugués: «Himno Nacional Brasileño»)
¿Problemas al reproducir este archivo?
Capital
Brasilia
22°54′40″S 43°12′20″O
Ciudad más
poblada
São Paulo
23°33′S 46°38′O
Idioma oficial
Portugués
 • Hablados
Lenguas indígenas de
Brasil
Gentilicio
Brasileño, -

In [3]:
pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 35.5 MB/s eta 0:00:00


In [11]:
# Ruta del archivo CSV
csv_file = 'WorldPopulation2023.csv'

# Cargar datos desde el CSV
population_data = pd.read_csv(csv_file)

# Mostrar los primeros registros para verificar la carga correcta
print("Primeros registros del CSV:")
print(population_data.head())


Primeros registros del CSV:
   Rank         Country  Population2023 YearlyChange  NetChange  \
0    36     Afghanistan        42239854       2.70 %    1111083   
1   138         Albania         2832439      -0.35 %      -9882   
2    34         Algeria        45606480       1.57 %     703255   
3   212  American Samoa           43914      -0.81 %       -359   
4   202         Andorra           80088       0.33 %        264   

   Density(P/Km²)  Land Area(Km²)  Migrants(net)  Fert.Rate  MedianAge  \
0              65          652860         -65846        4.4       17.0   
1             103           27400          -8000        1.4       38.0   
2              19         2381740          -9999        2.8       28.0   
3             220             200           -790        2.2       29.0   
4             170             470            200        1.1       43.0   

  UrbanPop% WorldShare  
0      26 %     0.53 %  
1      67 %     0.04 %  
2      75 %     0.57 %  
3      N.A.     0.00 %  

In [30]:
import requests

# Función para hacer una consulta a Wikidata
def query_wikidata(query):
    url = "https://query.wikidata.org/sparql"
    headers = {
        'User-Agent': 'NLP_FINAL',
        'Accept': 'application/json'
    }
    params = {
        'query': query,
        'format': 'json'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

# Consulta SPARQL para obtener el PIB de varios países de América del Sur
sparql_query = """
SELECT ?countryLabel ?gdpValue WHERE {
  ?country wdt:P31 wd:Q3624078;  # instance of country
           wdt:P154 ?gdpValue.  # GDP value
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

# Ejecutar la consulta
results = query_wikidata(sparql_query)

# Procesar los resultados
for result in results['results']['bindings']:
    country_name = result['countryLabel']['value']
    gdp_value = result['gdpValue']['value']

    # Aquí podrías almacenar estos datos en Neo4j, utilizando transacciones Cypher
    print(f"País: {country_name}, PIB: {gdp_value}")


País: Canada, PIB: http://commons.wikimedia.org/wiki/Special:FilePath/Canada%20wordmark.svg
País: Switzerland, PIB: http://commons.wikimedia.org/wiki/Special:FilePath/Logo%20der%20Schweizerischen%20Eidgenossenschaft.svg
País: Portugal, PIB: http://commons.wikimedia.org/wiki/Special:FilePath/Log%C3%B3tipo%20do%20Governo%20Portugal.svg
País: France, PIB: http://commons.wikimedia.org/wiki/Special:FilePath/Logo%20RF.svg


In [31]:
from neo4j import GraphDatabase

# Reemplaza con la URL `neo4j+s` de tu instancia
uri = "neo4j+s://8078a6fb.databases.neo4j.io"
username = "neo4j"
password = "IlvVpDxEfEtgkP8_CJOXXjAB2PoMhE3yA49DPTD2FyU"

# Función para insertar datos en Neo4j
def insert_data_to_neo4j(country_name, gdp_value):
    query = """
    MERGE (country:Country {name: $country_name})
    SET country.gdp = $gdp_value
    """
    with driver.session() as session:
        session.run(query, country_name=country_name, gdp_value=gdp_value)

# Conectar a Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Procesar y almacenar los resultados
for result in results['results']['bindings']:
    country_name = result['countryLabel']['value']
    gdp_value = result['gdpValue']['value']

    # Insertar datos en Neo4j
    insert_data_to_neo4j(country_name, gdp_value)

# Cerrar la conexión a Neo4j al finalizar
driver.close()

In [33]:
pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69

In [35]:
import gensim.downloader as api
from gensim.models import Word2Vec

# Descargar y cargar el modelo Word2Vec preentrenado
model = api.load('word2vec-google-news-300')

# Ejemplo de procesamiento y almacenamiento de embeddings
processed_embeddings = []
for fragment in processed_texts:
    # Obtener el embedding promedio para cada fragmento de texto
    words = fragment.split()  # Dividir el fragmento en palabras
    embeddings = [model[word] for word in words if word in model]  # Obtener embeddings de las palabras en el modelo
    if embeddings:
        mean_embedding = sum(embeddings) / len(embeddings)  # Calcular el embedding promedio
        processed_embeddings.append(mean_embedding)

class ChromaDB:
    def insert_embedding(self, embedding):
        # Simulación de inserción en la base de datos
        print(f"Insertando embedding en ChromaDB: {embedding}")

# Crear instancia de ChromaDB (simulada)
chroma_db = ChromaDB()

# Insertar los vectores de embeddings en ChromaDB (ejemplo genérico, adaptar según ChromaDB)
for vector in processed_embeddings:
    chroma_db.insert_embedding(vector)

print("Embeddings almacenados en ChromaDB.")


[==================================================] 100.0% 1662.8/1662.8MB downloaded
Embeddings almacenados en ChromaDB.


In [38]:
import fitz  # Importar PyMuPDF para trabajar con PDFs
import pandas as pd
import numpy as np
import gensim.downloader as api
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Simulación de la clase ChromaDB
class ChromaDB:
    def __init__(self):
        self.embeddings = []

    def insert_embedding(self, embedding, metadata):
        # Simulación de inserción en la base de datos
        self.embeddings.append({"embedding": embedding, "metadata": metadata})
        print(f"Insertando embedding en ChromaDB: {embedding[:10]}... con metadata: {metadata}")

# Crear instancia de ChromaDB (simulada)
chroma_db = ChromaDB()

# Descargar y cargar el modelo Word2Vec preentrenado
model = api.load('word2vec-google-news-300')

# Función para obtener el embedding promedio de un texto
def get_mean_embedding(text):
    words = text.split()  # Dividir el fragmento en palabras
    embeddings = [model[word] for word in words if word in model]  # Obtener embeddings de las palabras en el modelo
    if embeddings:
        return np.mean(embeddings, axis=0)  # Calcular el embedding promedio
    return None



# Procesar e insertar embeddings de PDFs en ChromaDB
for pdf_file, chunks in processed_texts.items():
    for chunk in chunks:
        embedding = get_mean_embedding(chunk)
        if embedding is not None:
            chroma_db.insert_embedding(embedding.tolist(), {"pdf_file": pdf_file, "text": chunk})

# Procesar e insertar embeddings del CSV en ChromaDB
for index, row in population_data.iterrows():
    text = ' '.join(row.astype(str))  # Combinar todos los valores de la fila en un solo string
    embedding = get_mean_embedding(text)
    if embedding is not None:
        chroma_db.insert_embedding(embedding.tolist(), {"csv_file": csv_file, "row_index": index, "text": text})

print("Embeddings almacenados en ChromaDB.")

# Mostrar el contenido almacenado en ChromaDB
for item in chroma_db.embeddings:
    if 'pdf_file' in item['metadata']:
        print(f"PDF: {item['metadata']['pdf_file']}")
    if 'csv_file' in item['metadata']:
        print(f"CSV: {item['metadata']['csv_file']}, Row: {item['metadata']['row_index']}")
    print(f"Texto: {item['metadata']['text'][:200]}...")  # Mostrar solo los primeros 200 caracteres del texto
    print()

Se han truncado las últimas 5000 líneas del flujo de salida.
La producción de automóviles se incrementó desde los 169 621 vehículos fabricados en 2003 al récord histór...

PDF: Argentina.pdf
Texto: e Buenos
Aires; (celeste) y Patagonia (azul).
Desde el 2003 se duplicó el PIB industrial, al registrar un aumento del 105 %, con una fuerte suba de la productividad laboral. Se logró además un crecimi...

PDF: Argentina.pdf
Texto: toria de la industria en la Argentina
El turismo en Argentina es, con 6  759  000 turistas en 2017 según la Organización Mundial del Turismo, el país más
visitado de Sudamérica y el segundo más visita...

PDF: Argentina.pdf
Texto: e, desde la selva tropical de las yungas al norte hasta los valles, glaciares, lagos y bosques fríos
de la Patagonia Andina en el sur hasta la Antártida. Las gigantescas distancias exigen en la mayorí...

PDF: Argentina.pdf
Texto: visas.455 ​
 Los extranjeros reconocen a la Argentina como una zona libre de
conflictos armados, terrorismo o